In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn
from lenskit import topn
import pandas as pd
%matplotlib inline

In [4]:
ratings = pd.read_csv('dataSets/jester_dataset_2/jester_ratings_formated.csv', skiprows=1, names=['user', 'item', 'rating'])
ratings.head()

,user,item,rating
0,1,5,0.219
1,1,7,-9.281
2,1,8,-9.281
3,1,13,-6.781
4,1,15,0.875


In [5]:
ratings.rating += 10
print(ratings)

          user  item  rating
0            1     5  10.219
1            1     7   0.719
2            1     8   0.719
3            1    13   3.219
4            1    15  10.875
5            1    16   0.344
6            1    17   0.969
7            1    18   2.531
8            1    19   1.281
9            1    20   0.844
10           1    21   2.812
11           1    22   1.219
12           1    23   1.469
13           1    24   2.094
14           1    25   2.531
15           1    89  19.812
16           1    50  19.906
17           1   102  10.750
18           1   103   5.000
19           1   104  12.938
20           1   105  12.000
21           1   106   9.844
22           1   107  12.031
23           1   108  15.688
24           1   109  19.656
25           1    87  18.000
26           1    93  19.312
27           1    76  19.312
28           1    65  18.781
29           1    72  18.781
...        ...   ...     ...
1048546  39341   106  17.250
1048547  39341    82  16.906
1048548  39341

In [6]:
algo_ii = knn.ItemItem(20)
algo_als = als.BiasedMF(50)

In [7]:
def eval(aname, algo, train, test):
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    # now we run the recommender
    recs = batch.recommend(fittable, users, 10)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [8]:
all_recs = []
test_data = []
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
    test_data.append(test)
    all_recs.append(eval('ItemItem', algo_ii, train, test))

In [9]:
all_recs = pd.concat(all_recs, ignore_index=True)
print(all_recs)

        item      score   user  rank Algorithm
0         35   6.974737      3     1  ItemItem
1        129   6.963921      3     2  ItemItem
2         72   6.869047      3     3  ItemItem
3         53   6.823492      3     4  ItemItem
4         50   6.759644      3     5  ItemItem
5         62   6.747216      3     6  ItemItem
6        104   6.694772      3     7  ItemItem
7         68   6.544909      3     8  ItemItem
8         89   6.508654      3     9  ItemItem
9         32   6.489921      3    10  ItemItem
10       104  11.995918     29     1  ItemItem
11       105  11.870688     29     2  ItemItem
12       106  11.868314     29     3  ItemItem
13        53  11.846710     29     4  ItemItem
14       129  11.823658     29     5  ItemItem
15        35  11.761600     29     6  ItemItem
16       126  11.747223     29     7  ItemItem
17       114  11.594355     29     8  ItemItem
18        50  11.555082     29     9  ItemItem
19        49  11.422507     29    10  ItemItem
20       122 

In [10]:
test_data = pd.concat(test_data, ignore_index=True)

In [11]:
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg)
results = rla.compute(all_recs, test_data)
print(results)

/home/user/anaconda3/lib/python3.7/site-packages/lenskit/metrics/topn.py:104: RuntimeWarning: invalid value encountered in double_scalars
  return achieved / ideal


                     ndcg
user  Algorithm          
1     ItemItem   0.171626
2     ItemItem   0.092933
3     ItemItem   0.000000
4     ItemItem   0.000000
5     ItemItem   0.000000
6     ItemItem   0.335677
7     ItemItem   0.151409
8     ItemItem   0.300445
9     ItemItem   0.004761
10    ItemItem   0.255973
11    ItemItem   0.000000
12    ItemItem   0.000000
13    ItemItem   0.424505
14    ItemItem   0.000000
15    ItemItem   0.000000
16    ItemItem   0.133010
17    ItemItem   0.428256
18    ItemItem   0.000000
19    ItemItem   0.000000
22    ItemItem   0.100797
23    ItemItem   0.450610
24    ItemItem   0.000000
25    ItemItem   0.107793
26    ItemItem   0.000000
27    ItemItem   0.000000
28    ItemItem   0.000000
29    ItemItem   0.000000
33    ItemItem   0.392178
35    ItemItem   0.000000
37    ItemItem   0.419494
...                   ...
39310 ItemItem   0.000000
39311 ItemItem   0.356652
39312 ItemItem   0.000000
39313 ItemItem   0.000000
39314 ItemItem   0.564999
39315 ItemIt

In [12]:
results.groupby('Algorithm').ndcg.mean()

Algorithm
ItemItem    0.156411
Name: ndcg, dtype: float64

In [13]:
with open("Output.txt", "w") as text_file:
    print(f"Purchase Amount: {results}", file=text_file)


In [14]:
predict_for_user(user, items, ratings=None)

NameError: name 'predict_for_user' is not defined